In [1]:
import os

In [2]:
%pwd

'd:\\SimpliLearn\\Deep Learning\\Plant Disease Classification\\plant_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\SimpliLearn\\Deep Learning\\Plant Disease Classification\\plant_disease_classification'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from plant_disease_classification.constants import *
from plant_disease_classification.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [19]:
import os
import urllib.request as request
import zipfile
from plant_disease_classification import logger
from plant_disease_classification.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-11 00:33:27,771: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-11 00:33:27,816: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-11 00:33:27,845: INFO: common: created directory at: data]
[2023-10-11 00:33:27,869: INFO: common: created directory at: data/data_ingestion]
[2023-10-11 00:50:24,587: INFO: 4037744573: data/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 689512690
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "31503970274c82dd26ddb1decb8cf3e7d49bc0420b4281b23abc0201bffce097"
Last-Modified: Tue, 10 Oct 2023 19:01:09 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A6EC:382EDF:180EBD:1DE9C4:65259F75
Accept-Ranges: bytes
Date: Tue, 10 Oct 2023 19:03:28 GMT
Via: 